In [100]:
import requests
import json
from plaid import Client
import os
import codecs
import pandas as pd
from pandas.io.json import json_normalize
from json.decoder import JSONDecodeError
import numpy as np
import pygsheets
import google_auth_httplib2

In [64]:
def CreateItem (YOUR_PUBLIC_KEY, YOUR_INSTITUTION_ID):
    """
    This function creates item for transactions of the Bank of America and gets public_token
    """
    

    url = "https://sandbox.plaid.com/sandbox/public_token/create"

    payload = "{\n\t\"public_key\": " + '\"' + YOUR_PUBLIC_KEY + '\"' + ",\n\t\"institution_id\":" + '\"' + YOUR_INSTITUTION_ID + '\"' +  ",\n\t\"initial_products\": [\"transactions\"],\n\t\"options\": {\n\t\t\"webhook\": \"https://www.randomdomain.com/webhook_url\"\n\t}\n}"
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data = payload)
    res = json.loads(response.text.encode('utf8')) 
    
    return res


In [103]:
# input data

YOUR_PUBLIC_KEY = ""
YOUR_INSTITUTION_ID = 'ins_1'
YOUR_SECRET = ''
YOUR_CLIENT_ID = ''
jsonFileFromGoogle = '' # full path "C:/...."

In [101]:
# Create an item for transactions of the Bank of America and gets public_token

plaid_item = CreateItem(YOUR_PUBLIC_KEY, YOUR_INSTITUTION_ID)
YOUR_PUBLIC_TOKEN = plaid_item['public_token']

In [79]:
# Exchange public token for access token (for transactions of the Bank of America)

client = Client(client_id = YOUR_CLIENT_ID, secret=YOUR_SECRET, 
                public_key = YOUR_PUBLIC_KEY, environment='sandbox')
response = client.Item.public_token.exchange(YOUR_PUBLIC_TOKEN)

YOUR_ACCESS_TOKEN = response['access_token']

In [80]:
# Retrieve transactions

client = Client(client_id = YOUR_CLIENT_ID, secret = YOUR_SECRET, 
                public_key = YOUR_PUBLIC_KEY, environment='sandbox')

response = client.Transactions.get(YOUR_ACCESS_TOKEN, start_date='2020-01-01', end_date='2020-05-01')
transactions = response['transactions']

# the transactions in the response are paginated, so make multiple calls while increasing the offset to
# retrieve all transactions
while len(transactions) < response['total_transactions']:
    response = client.Transactions.get(YOUR_ACCESS_TOKEN, start_date='2020-01-01', end_date='2020-05-01',
                                       offset=len(transactions)
                                      )
    transactions.extend(response['transactions'])

In [87]:
# Extract data from json into Pandas DataFrames
accounts_data = pd.json_normalize(data = response['accounts'])
transactions_data = pd.json_normalize(data = response['transactions'])

# Join Pandas DataFrames into the single DataFrame
rawdata_table = pd.merge(transactions_data, accounts_data,  on = 'account_id')

In [89]:
# Prepare data to make cashflow report

rawdata_table['date'] = rawdata_table['date'].astype('datetime64[ns]')
rawdata_table['type_cashflow'] = ''

type_cashflowIndex = rawdata_table.columns.get_loc('type_cashflow')
amountIndex = rawdata_table.columns.get_loc('amount')
categoryIndex = rawdata_table.columns.get_loc('category')

for i in range(rawdata_table.shape[0]):
    listtostr = (rawdata_table.iat[i,categoryIndex])
    rawdata_table.iloc[[i],[categoryIndex]] = ', '.join(listtostr)
    
    if rawdata_table.iat[i,amountIndex] < 0:
        #print (joinedtable.iat[i,amountIndex])
        rawdata_table.iloc[[i],[type_cashflowIndex]] = 'Expense'
        
    elif rawdata_table.iat[i,amountIndex] >= 0:
        #print (joinedtable.iat[i,amountIndex])
        rawdata_table.iloc[[i],[type_cashflowIndex]] = 'Income'


In [98]:
# Make cashflow table
cashflow_table = pd.pivot_table(rawdata_table, index = ['type_cashflow', 'subtype', 'category'], 
                                values = ['amount'], 
                                aggfunc = {'amount' : np.sum}, 
                                columns = pd.Grouper(freq='M', key='date'))
cashflow_table = cashflow_table.reset_index()

In [115]:
#authorization
gc = pygsheets.authorize(service_file = jsonFileFromGoogle)

#open the google spreadsheet
sh = gc.open('xO Analytics')

#select the first and second sheets 
sheet_1 = sh[0]
sheet_2 = sh[1]

#update the first sand second sheets with rawdata and cashflow_tables. 
sheet_1.set_dataframe(rawdata_table,(1,1))
sheet_2.set_dataframe(cashflow_table,(1,1))